In [31]:
import pandas as pd

df = pd.read_csv('card_transdata.csv').sample(50000)
df

distance_from_home  distance_from_last_transaction  \
263799            1.485610                        7.348750   
335382            8.656956                        1.525141   
151220           52.408527                        0.177572   
435671           24.034953                        2.410045   
565303            6.747112                        5.976245   
...                    ...                             ...   
55900             7.720786                        9.855437   
676185           12.507610                        7.166189   
899582           12.798826                        0.191575   
69177           127.018599                        1.635782   
659982           49.636322                        6.854095   

        ratio_to_median_purchase_price  repeat_retailer  used_chip  \
263799                        0.458636              0.0        0.0   
335382                        3.477582              1.0        0.0   
151220                        7.115511              1.0        0.0   
435671                        0.490536              1.0        0.0   
565303                        0.167656              1.0        0.0   
...                                ...              ...        ...   
55900                         0.743087              1.0        0.0   
676185                        2.919751              1.0        1.0   
899582                        3.562658              1.0        1.0   
69177                         0.604155              1.0        1.0   
659982                        0.311272              1.0        0.0   

        used_pin_number  online_order  fraud  
263799              0.0           0.0    0.0  
335382              1.0           0.0    0.0  
151220              0.0           1.0    1.0  
435671              0.0           0.0    0.0  
565303              0.0           1.0    0.0  
...                 ...           ...    ...  
55900               0.0           1.0    0.0  
676185              0.0           1.0    0.0  
899582              0.0           1.0    0.0  
69177               0.0           0.0    0.0  
659982              0.0           0.0    0.0  

[50000 rows x 8 columns]

In [32]:
import torch as pt
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [40]:
import torch.optim as optim
model = Model()


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

from sklearn.model_selection import train_test_split
y=pt.LongTensor(list(df['fraud']))
X=pt.FloatTensor(df.drop(['fraud'], axis=1).iloc[:, :-1].values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch + 1}/{10}], Loss: {loss.item():.4f}')

from sklearn.metrics import accuracy_score,r2_score , mean_squared_error
model.eval()
with pt.no_grad():
    outputs = model(X_test)
    _, predicted = pt.max(outputs.data, 1)
    accuracy = accuracy_score(y_test, predicted)
    print(f'Accuracy of the model on the test dataset: {accuracy:.2f}')

Epoch [1/10], Loss: 0.6803
Epoch [2/10], Loss: 0.6736
Epoch [3/10], Loss: 0.6674
Epoch [4/10], Loss: 0.6618
Epoch [5/10], Loss: 0.6564
Epoch [6/10], Loss: 0.6512
Epoch [7/10], Loss: 0.6463
Epoch [8/10], Loss: 0.6417
Epoch [9/10], Loss: 0.6374
Epoch [10/10], Loss: 0.6335
Accuracy of the model on the test dataset: 0.91


C:\Users\1\AppData\Local\Temp\ipykernel_11108\4126805864.py:9: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y=pt.LongTensor(list(df['fraud']))
